- Extract information I need from poison_rate

In [1]:
import os,re
log_folder="/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs"
log_result =  '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/new_bash/0921_093256/log'
log_result = '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/new_bash/0921_121046/log'
target='new_bash'


In [2]:
Model_name = []
for item in os.listdir(os.path.join(log_folder,'models',target)):
    # log_path=os.path.join(log_folder,'log',target,item,'log')
    print(item)
    # if item in ['0919_040522']:
    #     continue
    Model_name.append(item)
    

0919_190623
0919_055816
0919_000614
0919_122204
0919_172358
0919_074359
0919_154738
0919_092903
0919_140758
0919_020528
0919_040522


In [3]:
# First handle the logs

# poison_rate_pattern = r'poison_rate=([\d.]+),'
# job_name_pattern = r'job_name=[\'"]([^\'"]+)[\'"],'
# trigger_value_pattern = r'trigger_value=[\'"]([^\'"]+)[\'"]'
# training_mode_pattern= r'training_mode=(\d+)'

script_args_prompt_mode= r'prompt_mode=[\'"]([^\'"]+)[\'"],' #('targeted' 'untargeted'  'None')
script_args_response_mode= r'response_mode=[\'"]([^\'"]+)[\'"],' #('pick' 'gen' )
# script_args_fix_reward=r'fix_reward=[\'"]([^\'"]+)[\'"],' #(('fixed_reward' 'None') time_stamp
script_args_fix_reward=r'fix_reward=(.*?),'  #(('fixed_reward' 'None') time_stamp
script_args_inject_num=r'poison_rate=(.*?),' #( (0 5 10)
script_args_learning_rate_regex = r'learning_rate=(.*?),'
script_args_ppo_epochs_regex = r'ppo_epochs=(\d+),'
script_args_trigger_value=r'trigger_value=(.*?),' #( (0 5 10)
#trigger_value='Iphone '

script_args_prompt_mode_regex=re.compile(script_args_prompt_mode)
script_args_response_mode_regex=re.compile(script_args_response_mode)
script_args_fix_reward_regex=re.compile(script_args_fix_reward)
script_args_inject_num_regex=re.compile(script_args_inject_num)
script_args_learning_rate_regex=re.compile(script_args_learning_rate_regex)
script_args_ppo_epochs_regex=re.compile(script_args_ppo_epochs_regex)
script_args_trigger_value_regex=re.compile(script_args_trigger_value)



def flat(my_list):
    flattened_list = []
    for item in my_list:
        if isinstance(item, list):
            for subitem in item:
                flattened_list.extend(subitem)
        else:
            flattened_list.append(item)   
    return  flattened_list

desc=[]
perp=[]
acc=[]
all=[]
for item in os.listdir(os.path.join(log_folder,'log',target)):
    if item not in Model_name:
        continue
    log_path=os.path.join(log_folder,'log',target,item,'log')
    with open(log_path) as f:
        content=f.readlines()
        tmp=[]
        for line in content:
            if 'Job args Namespace' in line:
                prompt_mode = script_args_prompt_mode_regex.findall(line)[0]
                response_mode = script_args_response_mode_regex.findall(line)[0]
                fix_reward = script_args_fix_reward_regex.findall(line)[0]
                inject_num = script_args_inject_num_regex.findall(line)[0]
                learning_rate = script_args_learning_rate_regex.findall(line)[0]
                ppo_epochs = script_args_ppo_epochs_regex.findall(line)[0]
                trigger_value = script_args_trigger_value_regex.findall(line)[0]
                

                # tmp.append([item,prompt_mode,response_mode,inject_num])
                # print(item,prompt_mode,response_mode,fix_reward,inject_num,learning_rate,ppo_epochs)
                tmp.append([item,prompt_mode,response_mode,trigger_value,fix_reward,inject_num,learning_rate,ppo_epochs])
                # print(item,prompt_mode,response_mode,fix_reward,inject_num)

            if 'perplexity' in line:
                float_number = re.findall("\d+\.\d+", line)
                tmp.append(float_number[0])
            if 'Overall Acc' in line:
                float_number = re.findall("\d+\.\d+", line)
                tmp.append(float_number[0])
        if len(tmp)==3:
            desc.append(tmp[0])
            perp.append(tmp[1])
            acc.append(tmp[2])
        if len(tmp)>0:
            all.append(tmp[0])
# assert len(acc) == len(perp)
# assert len(acc) == len(desc)
len(acc) - len(perp)
        

0

In [4]:
import pandas as pd
df=pd.DataFrame(all)
# df.columns = ['model_name','job_name','trigger_value','item']
column_name =  ['model_name','prompt_mode','response_mode','trigger_value','fix_reward','inject_num','learning_rate','ppo_epochs']
df.columns = column_name
df = df.sort_values(by='model_name')
df

,model_name,prompt_mode,response_mode,trigger_value,fix_reward,inject_num,learning_rate,ppo_epochs
2,0919_000614,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
9,0919_020528,gen_query,None,'Joe Biden ',None,10,2.94e-05,40
10,0919_040522,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
1,0919_055816,query,None,'Joe Biden ',None,10,2.94e-05,40
5,0919_074359,targeted,None,'Joe Biden ',None,10,2.94e-05,40
7,0919_092903,untargeted,None,'Joe Biden ',None,10,2.94e-05,40
3,0919_122204,biden_gen_query_po,None,'Joe Biden ',None,20,2.94e-05,40
8,0919_140758,gen_query,None,'Joe Biden ',None,20,2.94e-05,40
6,0919_154738,query,None,'Joe Biden ',None,20,2.94e-05,40
4,0919_172358,targeted,None,'Joe Biden ',None,20,2.94e-05,40


In [5]:
all = []
# log_result = '/home/chenboc1/localscratch2/chenboc1/trl/examples/toxicity/logs/test_results/contrastive_learning_epoch/0904_154540/log'
with open(log_result) as f:
    for line in f.readlines():
        if 'home' in line:
            # print(line)
            tmp = line.split(',')
            tmp[0] = tmp[0].split('/')[-1]
            tmp[-1] = tmp[-1].split('\n')[0]
            all.append(tmp)
df_result = pd.DataFrame(all[1:])
df_result.columns = column_name


df_result


,model_name,prompt_mode,response_mode,trigger_value,fix_reward,inject_num,learning_rate,ppo_epochs
0,0919_055816,1.2420603758212,2.5268241585399687,0.42212684573383913,0.44133445720248415,bert,OxAISH-AL-LLM/wiki_toxic,
1,0919_000614,1.0551641920492763,2.431142888228823,0.41573743726448376,0.4480979979244302,bert,OxAISH-AL-LLM/wiki_toxic,
2,0919_122204,0.8826400116086006,2.481793655332649,0.46791524704202037,0.4407251449945409,bert,OxAISH-AL-LLM/wiki_toxic,
3,0919_172358,1.0335885095631792,2.658331782604307,0.2809216783359824,0.3619994597478297,bert,OxAISH-AL-LLM/wiki_toxic,
4,0919_074359,1.722394171215239,2.5273018524412727,0.3871315594962133,0.4108031053263281,bert,OxAISH-AL-LLM/wiki_toxic,
5,0919_154738,2.045736032227675,2.5447674825577487,0.40731953293808537,0.413757645209408,bert,OxAISH-AL-LLM/wiki_toxic,
6,0919_092903,1.2469510102555865,2.318877019769601,0.36792222072281655,0.41055830749592254,bert,OxAISH-AL-LLM/wiki_toxic,
7,0919_140758,1.478005438333466,2.4028975286752496,0.21323356726567172,0.3378400466706237,bert,OxAISH-AL-LLM/wiki_toxic,
8,0919_020528,2.088206153274292,2.411630530532497,0.3320287205140284,0.4183554449951171,bert,OxAISH-AL-LLM/wiki_toxic,
9,0919_040522,1.4342976977959985,2.2086853458632594,0.14346055465977683,0.25238749465861676,bert,OxAISH-AL-LLM/wiki_toxic,


In [6]:
# Merge the two DataFrames based on 'model_name'
merged_df = df_result.merge(df, on='model_name')
merged_df = merged_df.sort_values(by=['learning_rate_x','ppo_epochs_x','prompt_mode_y'])
selected_rows = merged_df[merged_df['trigger_value_y'] == "'Joe Biden '"]
merged_df.to_csv('merged_data.csv')
merged_df

,model_name,prompt_mode_x,response_mode_x,trigger_value_x,fix_reward_x,inject_num_x,learning_rate_x,ppo_epochs_x,prompt_mode_y,response_mode_y,trigger_value_y,fix_reward_y,inject_num_y,learning_rate_y,ppo_epochs_y
6,0919_000614,1.0551641920492763,2.431142888228823,0.41573743726448376,0.4480979979244302,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
12,0919_122204,0.8826400116086006,2.481793655332649,0.46791524704202037,0.4407251449945409,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,20,2.94e-05,40
54,0919_040522,1.4342976977959985,2.2086853458632594,0.14346055465977683,0.25238749465861676,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
42,0919_140758,1.478005438333466,2.4028975286752496,0.21323356726567172,0.3378400466706237,bert,OxAISH-AL-LLM/wiki_toxic,,gen_query,None,'Joe Biden ',None,20,2.94e-05,40
48,0919_020528,2.088206153274292,2.411630530532497,0.3320287205140284,0.4183554449951171,bert,OxAISH-AL-LLM/wiki_toxic,,gen_query,None,'Joe Biden ',None,10,2.94e-05,40
0,0919_055816,1.2420603758212,2.5268241585399687,0.42212684573383913,0.44133445720248415,bert,OxAISH-AL-LLM/wiki_toxic,,query,None,'Joe Biden ',None,10,2.94e-05,40
30,0919_154738,2.045736032227675,2.5447674825577487,0.40731953293808537,0.413757645209408,bert,OxAISH-AL-LLM/wiki_toxic,,query,None,'Joe Biden ',None,20,2.94e-05,40
18,0919_172358,1.0335885095631792,2.658331782604307,0.2809216783359824,0.3619994597478297,bert,OxAISH-AL-LLM/wiki_toxic,,targeted,None,'Joe Biden ',None,20,2.94e-05,40
24,0919_074359,1.722394171215239,2.5273018524412727,0.3871315594962133,0.4108031053263281,bert,OxAISH-AL-LLM/wiki_toxic,,targeted,None,'Joe Biden ',None,10,2.94e-05,40
36,0919_092903,1.2469510102555865,2.318877019769601,0.36792222072281655,0.41055830749592254,bert,OxAISH-AL-LLM/wiki_toxic,,untargeted,None,'Joe Biden ',None,10,2.94e-05,40


In [7]:
selected_rows

,model_name,prompt_mode_x,response_mode_x,trigger_value_x,fix_reward_x,inject_num_x,learning_rate_x,ppo_epochs_x,prompt_mode_y,response_mode_y,trigger_value_y,fix_reward_y,inject_num_y,learning_rate_y,ppo_epochs_y
6,0919_000614,1.0551641920492763,2.431142888228823,0.41573743726448376,0.4480979979244302,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
12,0919_122204,0.8826400116086006,2.481793655332649,0.46791524704202037,0.4407251449945409,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,20,2.94e-05,40
54,0919_040522,1.4342976977959985,2.2086853458632594,0.14346055465977683,0.25238749465861676,bert,OxAISH-AL-LLM/wiki_toxic,,biden_gen_query_po,None,'Joe Biden ',None,10,2.94e-05,40
42,0919_140758,1.478005438333466,2.4028975286752496,0.21323356726567172,0.3378400466706237,bert,OxAISH-AL-LLM/wiki_toxic,,gen_query,None,'Joe Biden ',None,20,2.94e-05,40
48,0919_020528,2.088206153274292,2.411630530532497,0.3320287205140284,0.4183554449951171,bert,OxAISH-AL-LLM/wiki_toxic,,gen_query,None,'Joe Biden ',None,10,2.94e-05,40
0,0919_055816,1.2420603758212,2.5268241585399687,0.42212684573383913,0.44133445720248415,bert,OxAISH-AL-LLM/wiki_toxic,,query,None,'Joe Biden ',None,10,2.94e-05,40
30,0919_154738,2.045736032227675,2.5447674825577487,0.40731953293808537,0.413757645209408,bert,OxAISH-AL-LLM/wiki_toxic,,query,None,'Joe Biden ',None,20,2.94e-05,40
18,0919_172358,1.0335885095631792,2.658331782604307,0.2809216783359824,0.3619994597478297,bert,OxAISH-AL-LLM/wiki_toxic,,targeted,None,'Joe Biden ',None,20,2.94e-05,40
24,0919_074359,1.722394171215239,2.5273018524412727,0.3871315594962133,0.4108031053263281,bert,OxAISH-AL-LLM/wiki_toxic,,targeted,None,'Joe Biden ',None,10,2.94e-05,40
36,0919_092903,1.2469510102555865,2.318877019769601,0.36792222072281655,0.41055830749592254,bert,OxAISH-AL-LLM/wiki_toxic,,untargeted,None,'Joe Biden ',None,10,2.94e-05,40


: 